In [1]:
# Import necessary modules
import numpy as np
import pandas as pd
from math import exp

In [2]:
# Initialize Network
def initialize_network(inputs, n_hiddens, n_hidden_neurons, n_outputs):
    # Create network with n hidden layers with each having n neurons
    network = {}
    network['input_layer'] = {}
        
    n = 1
    while(n <= n_hiddens):
        network['hidden_' + str(n) + '_layer'] = {}
        if n == 1:
            network['hidden_' + str(n) + '_layer']['weights']\
                                                    = np.random.random((n_hidden_neurons, len(inputs)+1))
        else:
            network['hidden_' + str(n) + '_layer']['weights']\
                                                = np.random.random((n_hidden_neurons, n_hidden_neurons+1))
        n += 1
    
    network['output_layer']= {} 
    network['output_layer']['weights'] = np.random.random((n_outputs, n_hidden_neurons+1))
    
    return network

In [3]:
# Calculate neuron
def calculate_neuron(weights, inputs):
    inputs_plus_bias = np.append(inputs, 1)
    outputs = np.dot(weights, inputs_plus_bias)
    
    return outputs

In [4]:
# Activate neuron using Sigmoid 
def activate_sigmoid(output):
    return 1.0 / (1.0 + exp(-output))

In [5]:
# Forward propagate inputs
def forward_propagate(network, inputs):
    for layer, values in network.items():
        if layer == 'input_layer':
            continue
         
        outputs = calculate_neuron(values['weights'], inputs)
        
        outputs_list = []  
        for output in outputs:
            sigmoid_output = activate_sigmoid(output)
            outputs_list.append(sigmoid_output)
        sigmoid_outputs = np.array(outputs_list)
        
        values['outputs'] = outputs
        values['sigmoid'] = sigmoid_outputs
        inputs = values['sigmoid']
        
    return inputs                 

In [5]:
# Forward propagate inputs
def forward_propagate(network, inputs):
    for layer, values in network.items():
        if layer == 'input_layer':
            values['outputs'] = inputs
        else:
            outputs = calculate_neuron(values['weights'], inputs)            
            outputs_list = []  
            for output in outputs:
                sigmoid_output = activate_sigmoid(output)
                outputs_list.append(sigmoid_output)
            sigmoid_outputs = np.array(outputs_list)
            
            values['outputs'] = outputs
            values['sigmoid'] = sigmoid_outputs
            inputs = values['sigmoid']
        
    return inputs                 

In [6]:
np.random.seed(2023)
inputs = np.array([14.88, 14.57, 0.8811, 5.554])
network = initialize_network(inputs, 3, 2, 3)

for key, value in network.items():
    print(key, value)

input_layer {}
hidden_1_layer {'weights': array([[0.3219883 , 0.89042245, 0.58805226, 0.12659609, 0.14134122],
       [0.46789559, 0.02208966, 0.72727471, 0.52438734, 0.54493524]])}
hidden_2_layer {'weights': array([[0.45637326, 0.50138226, 0.39446855],
       [0.1511723 , 0.36087518, 0.16207701]])}
hidden_3_layer {'weights': array([[0.33795869, 0.18032328, 0.3909914 ],
       [0.03564821, 0.56486165, 0.20346149]])}
output_layer {'weights': array([[0.32060446, 0.37656378, 0.18405414],
       [0.10395184, 0.45492722, 0.19586384],
       [0.37852542, 0.93053196, 0.76015971]])}


In [8]:
for layer, values in network.items():
    print(layer, values)

input_layer {'outputs': array([14.88  , 14.57  ,  0.8811,  5.554 ])}
hidden_1_layer {'weights': array([[0.3219883 , 0.89042245, 0.58805226, 0.12659609, 0.14134122],
       [0.46789559, 0.02208966, 0.72727471, 0.52438734, 0.54493524]]), 'outputs': array([19.12722985, 11.38231707]), 'sigmoid': array([1.       , 0.9999886])}
hidden_2_layer {'weights': array([[0.45637326, 0.50138226, 0.39446855],
       [0.1511723 , 0.36087518, 0.16207701]]), 'outputs': array([1.35221836, 0.67412037]), 'sigmoid': array([0.79449207, 0.66242517])}
hidden_3_layer {'weights': array([[0.33795869, 0.18032328, 0.3909914 ],
       [0.03564821, 0.56486165, 0.20346149]]), 'outputs': array([0.77894758, 0.60596228]), 'sigmoid': array([0.68545325, 0.6470192 ])}
output_layer {'weights': array([[0.32060446, 0.37656378, 0.18405414],
       [0.10395184, 0.45492722, 0.19586384],
       [0.37852542, 0.93053196, 0.76015971]]), 'outputs': array([0.6474575 , 0.56146461, 1.62169323]), 'sigmoid': array([0.65643729, 0.63679135, 0.

In [18]:
# Calculate the derivative of an neuron output
def transfer_derivative_sigmoid(output):
    return output * (1.0 - output)

In [19]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in range(len(network)-1, -1, -1):
        layer = network[i]
        errors, error_signals = [],[]
        
        for j, neuron in enumerate(layer):
            if i == len(network) -1:
                error = neuron["output"] - expected[j]
                error_signal = error * transfer_derivative_sigmoid(neuron["output"])
                
            else:
                error = 0
                for k, next_neuron in enumerate(network[i+1]):
                    error += next_neuron["weights"][j] * next_neuron["delta"]
                    error_signal = error * transfer_derivative_sigmoid(neuron["output"])
                    errors.append(error)
                    error_signals.append(error_signal) 
            
            errors.append(error)
            error_signals.append(error_signal) 
            neuron['delta'] = error_signal
        

In [20]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        if i == 0:
            inputs = row[:-1]
        else:
            inputs = [neuron['output'] for neuron in network[i-1]]
            
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron["weights"][j] -= l_rate * neuron['delta'] * inputs[j]
                neuron["weights"][-1] -= l_rate * neuron['delta']

In [22]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1   
            sum_error += sum([(expected[i] - outputs[i]) ** 2 for i in range(n_outputs)]) 
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print(f"epoch:{epoch:2d},  lrate:{l_rate},  error:{sum_error:.4f}")    

In [16]:
weights = np.array([[0.32060446, 0.37656378, 0.18405414],
       [0.10395184, 0.45492722, 0.19586384],
       [0.37852542, 0.93053196, 0.76015971]])
inputs = np.array([0.68545325, 0.6470192 , 1])
outputs = np.dot(weights, inputs)

print(outputs)

[0.6474575  0.56146461 1.62169323]


In [10]:
output = forward_propagate(network, inputs)
print()
print()
print(output)



[0.65643729 0.63679135 0.83502852]
